<a href="https://colab.research.google.com/github/mabittar/Portfolio/blob/master/PNGMart_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web Scraper 

Nesse web scraper  veremos como criar um algorítmo para baixar todas as imagens, de forma classificada do site: pngmart.com utilizando o [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/).

Esse é apenas um exemplo de aplicação de um Crawler, que pode ser utilizado para registrar preços de um produto, oportunidades de imóveis para venda ou locação, disponibilidade de servidores e assim por diante.

Web Crawler, bot ou web spider é um algoritmo utilizado para encontrar, ler e gravar informações de páginas da internet. É como um robô que varre o caminho indicado e captura as informações que encontra pela frente.

Um dos maiores exemplos de um web crawler é o próprio google. Antes do site estar disponível para pesquisa, um robô lê o web site e cataloga as informações de forma a serem recuperadas numa busca futura.


Para esse web scrapper dividi o script em funções específicas a fim de facilitar qualquer possível manutenção.




In [1]:
# importando as bibliotecas necessárias
import requests
from bs4 import BeautifulSoup

## O Alvo

In [2]:
# link para o site map
link = 'http://www.pngmart.com/sitemapindex.xml/'

O site está dividido em diversas subpáginas, uma alternativa para buscarmos em quais páginas as imagens estão listadas foi utilizar o site map.

In [5]:
# função para extrair as subpáginas que listam as imagens
def get_url(link):
  """
  essa função mapeia o sitemap e retorna as subpáginas onde as imagens estão listadas
  entrada: link para o sitemap
  saída: lista de páginas onde as imagens são listadas
  """
    response = requests.get(link)
    xml = response.text
    soup = BeautifulSoup(xml, 'xml')

    site_maps = []

    for loc in soup.find_all('loc'):
      url = loc.text
      if 'part' in url:
        site_maps.append(url)

    return site_maps

In [24]:
site_map = get_url(link)
print("Foram encontradas {} páginas onde as imagens estão distribuídas." .format(len(site_map)))

Foram encontradas 14 páginas onde as imagens estão distribuídas


Com a função `get_map` é possível obter todas as subpáginas onde as imagens são apresentadas.



## Obtendo acesso as imagens

Agora que temos acesso as subpáginas a iremos elaborar uma função para obter o link para todas as imagens.

In [19]:
def master_list(url):
  """
  essa função tem como parametros:
    entrada:  lista de subpáginas onde as imagens estão listas obtida na função site_map
    saída: lista de url das imagens que estão contidas nas subpáginas
  """  
    images_list = []
    for loc in soup.find_all('loc'):
      img_url = loc.text
      images_list.append(img_url)

    return images_list

In [ ]:
# exemplo para não sobrecarregarmos o site
site_map_1 = site_map[0]

In [25]:
# atribuindo a lista de imagens (nesse caso apenas um exemplo) o resultado obtido na  função site_map

img_list = master_list(site_map_1) #estou utilizando a página 1 como exemplo
print("Foram encontradas {} imagens por página." .format(len(img_list)))

Foram encontradas 5000 imagens por página.


Criamos a função `master_list` que recebe como entrada a lista de subpáginas da função `get_map`, a partir dessa lista a função irá percorrer todas as subpágians e listar todas as urls dda imagens disponíveis.

Nesse ponto criei a variável site_map_1 apenas para esse estudo, já que a não há necessidade de sobrecarregar o servidor nesse estudo.

Veja que para cada subpágina são listadas 5.000 imagens, no momento eu que eu executei esse notebook eram 14 subpáginas, portanto são aproximadamente 70.000 imagens disponíveis.

Para utilizar a função basta atribuir a uma variável a função passando como parâmetro a lista obtida no site_map.

## Obtendo as Imagens

Agora que estamos com a lista das imagens disponível iremos desenvolver uma função que obtenha as imagens, faça o donwload e renomeie o arquivo mantendo os dados originais do arquivo para posterior utilização.

In [27]:
def get_img(img_list, start=0, end=20):
  """
  essa função executa o download das imagens
  parametros 
    entrada:
      lista de imagens da função master_list
      para não saturar o servidor foi incluído um início e fim
        confira o len(img_list) para verificar quantas imagens há por lista
    saída:
      salva as imagens no diretorio onde o script está sendo executado
      as imagens serão salvas mantendo o nome original e o respectivo ID
  """
  for image in img_list[start:end]:
    response = requests.get(image)
    soup = BeautifulSoup(response.text, 'html.parser')
    png_link = soup.find('a', {'class': 'download'})['href']
    # obtendo o arquivo
    document = requests.get(png_link)
    # preservando o ID e nome do arquivo
    document_title = image.split('/')[-1] + '-' + png_link.split('/')[-1]

    # salvando os arquivos no diretório
    with open(document_title, 'wb') as file:
      file.write(document.content)

In [28]:
# utilizando a função get_img para realizar o donwload das imagens 1 a 9
get_img(img_list, 0, 10)

Com poucas linhas podemos realizar um scraper do site e obter todas as imagens listas.

Os próximos passos seria implementá-lo em um algorítmo Python para rodá-lo diretamente do computador. Pode-se ainda em pensar nos parâmetros de início e fim diretamente na função rum doo script.